In [ ]:
from os import getcwd
from common.utils import * 

input_dir = getcwd() + '/img/input_samples/'
outp_dir = getcwd() + '/img/output_fft_noise_images/'

figsize = (10, 4)
figsize_tall = (10, 8)
figsize_wide = (12, 4)
figsize_default = (6.4, 4.8)
cbar_loc = [0.9, 0.11, 0.02, 0.77]

SMALL_SIZE = 8
MEDIUM_SIZE = 14
BIGGER_SIZE = 16
FIGSIZE_WIDE = (9, 4.8)
FIGSIZE_TALL = (6.4, 6)
MARKERSIZE = 100

plt.rc('font', size=SMALL_SIZE)
plt.rc('axes', titlesize=BIGGER_SIZE)
plt.rc('axes', labelsize=BIGGER_SIZE)
plt.rc('xtick', labelsize=BIGGER_SIZE)
plt.rc('ytick', labelsize=BIGGER_SIZE)
plt.rc('legend', fontsize=MEDIUM_SIZE)
plt.rc('figure', titlesize=SMALL_SIZE)

%matplotlib inline

# FFT Spectrum leakage

In [ ]:
size = 100
x = freq_numbers_1d(size)
xx = adjust_spectrum_1d(x)
kernel = freq_pink_filter_1d(x)
kernel_sym = freq_pink_filter_1d(xx)

noise = np.random.normal(0, 1, size)
noise -= np.mean(noise)

zero_padded = np.copy(noise)
zero_padded[-1] = zero_padded[0] 
zero_padded -= np.mean(zero_padded)

noise_ft = find_ft_1d(noise)
noise_ft = adjust_spectrum_1d(noise_ft)
noise_magn_raw = np.abs(noise_ft)
noise_magn_n = normalize(noise_magn_raw)
noise_magn_l = np.log10(noise_magn_n)

zero_padded_ft = find_ft_1d(zero_padded)
zero_padded_ft = adjust_spectrum_1d(zero_padded_ft)
zero_padded_magn_raw = np.abs(zero_padded_ft)
zero_padded_magn_n = normalize(zero_padded_magn_raw)
zero_padded_magn_l = np.log10(zero_padded_magn_n)

ax = plot_graphs(noise, zero_padded, grid=True, figsize=figsize)
plt.show()

In [ ]:
ax = plot_graphs((xx, noise_magn_raw), (xx, zero_padded_magn_raw), grid=True, figsize=figsize)
plt.show()

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=figsize_wide)

ax1 = plot_graphs(noise, (zero_padded, '--'), ax=ax1, grid=True, ylabel='X(k)', xlabel='k, (а)')
ax2 = plot_graphs((xx, noise_magn_raw), (xx, zero_padded_magn_raw, '--'), ax=ax2, grid=True, ylabel='X(k)', xlabel='k, (б)')
# fig.savefig(outp_dir + 'spectrum_leakage.png')
plt.show()

# FFT Synthesis demo

In [ ]:
pink_noise_ft = noise_ft * kernel_sym
power_factor = normalize_psd(noise_magn_raw, np.abs(pink_noise_ft))
pink_noise = power_factor * find_ift_1d(pink_noise_ft)

ax = plot_graphs((x, noise), (x, kernel, 'g--'), (x, kernel * noise), grid=True)
plt.show()

In [ ]:
ax = plot_graphs(noise, (pink_noise, '--'), grid=True)
plt.show()

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=figsize_wide)
ax1 = plot_graphs(noise, (pink_noise, '--'), ax=ax1, ylabel='X(k)', xlabel='k, (а)', grid=True)
ax2 = plot_graphs((xx, noise_ft.real), (xx, 10 * kernel_sym, 'g--'), 
                  (xx, noise_ft.real * kernel_sym, '--'), ax=ax2,
                  ylabel='X(k)', xlabel='k, (б)', grid=True)
plt.subplots_adjust(wspace=0.3)
plt.show()

In [ ]:
plt.loglog(xx, noise_magn_raw)
plt.loglog(xx, kernel_sym)
plt.loglog(xx, kernel_sym * noise_magn_raw)
plt.gca().set_ybound(10 ** -3, 100)
plt.grid()
plt.show()

# 2D FFT Synthesis demo

In [ ]:
shape = (256, 256)
y_size, x_size = shape
xx, yy = freq_numbers_2d(shape)
x, y = freq_mesh_2d(shape)

noise = np.random.normal(0, 1, shape)
noise -= np.mean(noise)
noise_ft = find_ft_2d(noise)

kernel_1 = freq_pink_filter_2d(xx, yy, factor=1.5)
kernel_2 = freq_pink_filter_2d(xx, yy, factor=1.5, x_stretch=0.5)

pic_1 = find_ift_2d(noise_ft * kernel_1)
pic_1 = normalize_img(pic_1)

pic_2 = find_ift_2d(noise_ft * kernel_2)
pic_2 = normalize_img(pic_2)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=figsize)
im1 = ax1.imshow(pic_1, cmap='gray')
ax1.set_xlabel('а')

im2 = ax2.imshow(pic_2, cmap='gray')
ax2.set_xlabel('б')

cax = fig.add_axes([0.9, 0.11, 0.02, 0.77]) 
cbar = plt.colorbar(im2, cax=cax)

plt.subplots_adjust(wspace=0)
# plt.savefig(outp_dir + 'fourier_synthesis_2d_case.png')
plt.show()

In [ ]:
fig, ax = plt.subplots(subplot_kw={'projection': '3d'})
ax.plot_surface(x, y, np.log10(kernel_1))
ax.plot_surface(x, y, np.log10(kernel_2))
plt.show()

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=figsize)

im1 = ax1.imshow(np.log(kernel_1), cmap='gray')
ax1.set_xlabel('а')

im2 = ax2.imshow(np.log(kernel_2), cmap='gray')
ax2.set_xlabel('б')

cax = fig.add_axes([0.9, 0.11, 0.02, 0.77]) 
cbar = plt.colorbar(im2, cax=cax)

plt.subplots_adjust(wspace=0)
# plt.savefig(outp_dir + 'pink_noise_kernel_2d_case.png')
plt.show()

# Frequency domain Pink Noise shifting

In [ ]:
shape = (256, 256)
y_size, x_size = shape
xx, yy = freq_numbers_2d(shape)
x, y = freq_mesh_2d(shape)
dx, dy = 25, 20

# Generating cloud image
whitenoise = np.random.normal(0, 1, shape)
whitenoise_ft = find_ft_2d(whitenoise)
kernel = freq_pink_filter_2d(xx, yy, factor=2)
pink_ft = whitenoise_ft * kernel
pink_noise = normalize_img(find_ift_2d(pink_ft))

# Apply the phase shift along both axes
shift = np.exp(-1j * 2 * np.pi  * (x * dx / x_size + y * dy / y_size))
pink_ft_shifted = pink_ft * shift
shifted_pink_noise = normalize_img(find_ift_2d(pink_ft_shifted))

f, ax = show_images(pink_noise, shifted_pink_noise, figsize=figsize_tall)
plt.show()

# Smooth transition between cloud images

In [ ]:
shape = (256, 256)
y_size, x_size = shape
xx, yy = freq_numbers_2d(shape)
x, y = freq_mesh_2d(shape)
depth = int(x_size * 0.3)

img = gen_cloud(x_size, y_size)
img_new = make_img_transition_x(img, depth)

f, ax = show_images(img, img_new, figsize=figsize_tall)
plt.show()

In [ ]:
img = gen_cloud(x_size, y_size)
img_new = make_img_transition_y(img, depth)

f, ax = show_images(img, img_new, figsize=figsize_tall)
plt.show()

In [ ]:
img = gen_cloud(x_size, y_size)
img_row_1 = make_img_transition_x(img, depth)
img_row_2 = make_img_transition_y(img_row_1, depth)

f, ax = show_images(img_row_1, img_row_2, figsize=figsize_tall)
plt.show()

# Example of XY smooth transition and shift

In [ ]:
shape = (256, 256)
y_size, x_size = shape
xx, yy = freq_numbers_2d(shape)
dx, dy = 25, 20
depth = int(x_size * 0.3)

img = gen_cloud(x_size, y_size)
new_img = make_img_transition_xy(img, depth, outter_smooth=True)
shifted_img = shift_img_xy(new_img, dx, dy)

f, ax = show_images(img, new_img, shifted_img, figsize=figsize_tall)
plt.show()